# Conversion from SPG/SPA to CSV

Convert FTIR data given in OMNIC files (SPG/SPA files) into CSV files using [`spectrochempy` library](https://www.spectrochempy.fr/).

AIR and Background FTIR data are ignored since they vary across measurements and their magnitude is small compared to the material spectra so the method should be able to handle without further preprocessing steps.

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import spectrochempy as scp

"<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABgAAAAYCAYAAADgdz34AAAAAXNSR0IArs4c6QAAAAlw SFlzAAAJOgAACToB8GSSSgAAAetpVFh0WE1MOmNvbS5hZG9iZS54bXAAAAAAADx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6 bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDUuNC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIKICAgICAgICAgICAg eG1sbnM6eG1wPSJodHRwOi8vbnMuYWRvYmUuY29tL3hhcC8xLjAvIgogICAgICAgICAgICB4bWxuczp0aWZmPSJodHRwOi8vbnMu YWRvYmUuY29tL3RpZmYvMS4wLyI+CiAgICAgICAgIDx4bXA6Q3JlYXRvclRvb2w+bWF0cGxvdGxpYiB2ZXJzaW9uIDIuMS4wLCBo dHRwOi8vbWF0cGxvdGxpYi5vcmcvPC94bXA6Q3JlYXRvclRvb2w+CiAgICAgICAgIDx0aWZmOk9yaWVudGF0aW9uPjE8L3RpZmY6 T3JpZW50YXRpb24+CiAgICAgIDwvcmRmOkRlc2NyaXB0aW9uPgogICA8L3JkZjpSREY+CjwveDp4bXBtZXRhPgqNQaNYAAAGiUlE QVRIDY1We4xU1Rn/3XPuYx47u8w+hnU38hTcuoUEt/6D2y4RB0ME1BoEd9taJaKh9CFiN7YGp7appUAMNmktMZFoJTYVLVQ0smsy 26CN0SU1QgsuFAaW3WVmx33N677O6XfuyoIxTXqSO/fec+75fd93vt/3/UbDV0aKSZmCpkFMLz3T9utuu2N+o98aDSMBKVAo89z5 y+zEz3ZafcCOfvWdlGCalqKn1Bf71CygTd+mf1esSOnpdMpTb+vWpTZuWVfe3jLPa5tzHYNm0T5N0gpdkkHaDBeGBU6d1/t/fyS8 +/CbqdfUvmsx1PuMgc2bNxv79u1zgd31r+7JH1jbIZKxWRXAcYUQ8IWvBfBXNjEuJWPgMA02NR7C3/pYT9fjdZ3A9tGrWF8YSJHn qcDz3y7q2T967PZv+gnYJdd1mEZ+62zGDQV/dQgKhmLzDNOXCEWM3j6eTT5Y3w78dOBKJLR1PQf+4ivPj76UPZnssBN+wbM9Aet/ AV81Mf1EEULXYfOobvX2WWQk0aoioXwwSmirOlioY0mu8BIouzYl7P8GV3vpqCCEZvlFz769w08oLDWvyKIyL1asSm28d6WfzA97 ztvvV1kexUMsmhlkULEkuGYmFYC6AvfUrITnwUKl5K79lkjeSSRRTCTbQPd95e1WzMbZSya74XoXAxctCllCnbECMOjZNGRwvzIX nD85wbkMmKK+U045Dtdi8Qp+SAxU2GTg2bYlC9224pgvmSb54vkVTBQYyhUt2KjAMyMmPjwRQW5Mh2WKwJhlBh6jVGagFM84wZnQ 4bpC0Rt4pk1PbSt0NDcxDA5xryosDHWgtbM0DGZDWLSoiDMDYeQnGVrmOThxLozB0RAaahzkJzjKNqcIQBymJFMkOlN8Dqjpg0XY Tx5xO/QbmmUrqIjGJznq47TqTaClKYfjp+PInLMwnOdYvtQBZ2XcunQY+VwIo4U4muoFEjVEFE6lQyEUKzHYfgQG9ylCyngU+Cxj tOqxCDGHcCsOMCs6iQul5ZiStdATYxjMZXDLTUVwLY8Jey4uOh2IxjwsrP8UXJYxUrkZrghBahzV5iXU6gNkq0Z1EzIsUBUSCV2n EOHo0LVxHCpuxabJJdhi5PFnvw5vLXwXIfNZvD/+JNo/X40NegE54sUaazl+UL8XD1x+FB9Ijjt4EQfdGN6J/x131LwIV9ap/AYs 0x1fz1ZKFbh6A7qKy/By9Dg6G36Ep91vUJJ15Cqr0Z67E8/HzmBrw1OwxWyM+3Mo6BAuSB17oyfx0Oyl2DN0Hqs/70Cx6hBCvESF UY1ShWXZZEE7OTAYxZzaPH4TuoiusZvRnunFy2NbiHYuBp2vB66srX4vMEjpRKPxKXmnoQ4+Mn4DPiv8CYcrs3GfNUXJLtM+alSO hrMj/KT+wBNW3+E/2liywNO3iSflbaFva/+stGDTxE0E9Sjaox8HBhxpEamzMGSEaFKg+mjEddzDh1MxTDq3YV1kGBsjfwW3S9Cq anjmko+ndlb1UR3s6K8JlfphNWq9Ew/7c61T2BB/EbcaNkb8GBaE0tANH7/M34PLdhJDzjIcL9xPbdTG6zyM72Y+wXPHmvB489No fm0b5HnbQ9Rgp/7DSSd29AeVvPeNyK6JcYl/yQVi5dBjuGvoV/gaJe47s45QUxrDmcYX0MBsdF7egvXZ7+O0vZA4X8QmOQWjlSK7 RDz5wIM30gp9UbWcGjXxhzdDu1SiNSpx6kcQB57rPnr/3dlkZarWLnlRq5oPET1dOCIOk4wALib9eeS5iygfhkd09H0DWphB/+gs +PcOAS+ssrFmmXXgVfR0de9cpbAJfH3Q1jofW9DZk56dDcVsq9YcsoUMEd1qyLoT3BX1YiyHMJuk97hyjqIoE91t+NcTLeN0ZrfM oXatZbu6G0h4VG+ibqq0IJVK6cAjo6serG3vSUezCMct0yQeSOFJSUImqb2qbknUpDqlZxE0QZ+ZUpSlZx79h4Nda6zef9dlk121 JDjbR5XggPRZlRnS6bRQRtLpn4++cuie/Yvn2svmNxuLw9WCcYIl4fEoTEGiSTUqJdfgU+8ROqf1iMkLzS389YtNPXc/PH8l8ONB JZkHD+4JtD04HmVEDWWErmBhzV2/2LB1bemJG6krzv2S6NOHUgtEP0Oif5pE/3fHoruP7N8RiP61GArzSwbUhJJQpXJKiKbfr/3b IhKq76sKPUdF9NW/LSqfSn6vjv8C45H/6FSgvZQAAAAASUVORK5CYII=' style='height:25px; border-radius:12px; display:inline-block; float:left; vertical-align:middle'>",SpectroChemPy's API - v.0.4.1© Copyright 2014-2022 - A.Travert & C.Fernandez @ LCS


**User Param**

In [2]:
input_dir = 'Microplastics_BCET'
output_dir = 'Microplastics_BCET_csv'

## Convert SPG files to CSV

All data are given in SPG, except for Membrane filter data that is given in SPA.

In [3]:
fl = glob.glob(os.path.join(input_dir, '**', '*.spg'), recursive=True)

In [5]:
for f in fl:
    fn = os.path.basename(f)
    if fn.startswith('AIR') or fn.startswith('Membrane') :
        continue
    print(f'Reading: {f:60s} ...', end=' ')
    
    # prepare output folder
    subdir = os.path.split(os.path.dirname(f))[1]
    dir_out = os.path.join(output_dir, subdir)
    os.makedirs(dir_out, exist_ok=True)
    
    # read data and write each spectrum to csv file
    X = scp.read_omnic(f)
    for val, (_, name) in zip(X.data, X.y.labels):
        # print(val.shape, name)
        spectra = np.vstack([X.x.data, val]).T
        
        f_out = os.path.join(dir_out, name+'.csv')
        
        # spectra.tofile(f_out, sep = ',')
        pd.DataFrame(spectra).to_csv(f_out, header=None, index=False)
 
    print('done')

Reading: Microplastics_BCET\1mm-1NF1G-SB\1mm-1NF1G-SB.SPG             ... done
Reading: Microplastics_BCET\1mm-1NF2F-SB\1mm-1NF2F-SB.SPG             ... done
Reading: Microplastics_BCET\1mm-B1-SB\1mm-B1-SB.SPG                   ... done
Reading: Microplastics_BCET\1mm-B2-SB\1mm-B2-SB.SPG                   ... done
Reading: Microplastics_BCET\1mm-B3-SB\1mm-B3-SB.SPG                   ... done
Reading: Microplastics_BCET\1mm-EFF1G-SB\1mm-EFF1G-SB.SPG             ... done
Reading: Microplastics_BCET\1mm-EFF2F-SB\1 mm-EFF2F-SB.SPG            ... done
Reading: Microplastics_BCET\1mm-EFF2G-SB\1mm-EFF2G-SB.SPG             ... done
Reading: Microplastics_BCET\1mm-EFF3F-SB\1mm-EFF3F-SB.SPG             ... done
Reading: Microplastics_BCET\1mm-EFF3G-SB\1mm-EFF3G-SB.SPG             ... done
Reading: Microplastics_BCET\1mm-INF1F-SB\1mm-INF1F-SB.SPG             ... done
Reading: Microplastics_BCET\1mm-INF3G-SB\1mm-INF3G-SB.SPG             ... done
Reading: Microplastics_BCET\500um-B2-SB\500um-B2-SB.

## Convert SPA files to CSV

In [6]:
fl_membrane = glob.glob(os.path.join(input_dir, 'Membrane filter_SD', '**', 'Membrane*.spa'), recursive=True)

In [7]:
print(f'Reading: {"Membrane filter_SD":30s} ...', end=' ')

# prepare output folder
subdir = 'Membrane filter_SD'
dir_out = os.path.join(output_dir, subdir)
os.makedirs(dir_out, exist_ok=True)

# read data and write each spectrum to csv file
for f in fl_membrane:
    Y = scp.read_omnic(f)
    name = os.path.splitext(os.path.basename(f))[0]
    spectra = np.vstack([Y.x.data, Y.data]).T

    f_out = os.path.join(dir_out, name+'.csv')
    pd.DataFrame(spectra).to_csv(f_out, header=None, index=False)
print('done')

Reading: Membrane filter_SD             ... done
